# CNN Convolution Neural Network

cnn은 기본적으로 컨볼루션 계층(convolution layer, 합성곱 계층)과 풀링 계층(pooling layer)으로 구성된다. (2D 컨볼루션의 경우) 컨볼루션 계층과 풀링 계층 모두 2차원 평면 행렬에서 지정한 영역의 값들을 하나의 값으로 압축하는 것이다. 차이는 컨볼루션 계층은 가중치와 편향을 적용하여 하나의 값으로 압축하고, 풀링은 단순히 값중 하나를 선택해서 가져온다. 

- 윈도우 `Window`: 지정한 크기의 영역
- 스트라이드 `Stride`: 몇칸 씩 움직일지 여부
- 커널 또는 필터 `Kerner` or `Filter`: 컨볼루션을 적용할 때 사용할 윈도우 크기 만큼의 가중치와 1개의 편향
- 컨볼루션 `Convolution`: 윈도우 사이즈 만큼 가중치와 편향을 적용하여 압축
- 풀링 `Pooling`: 윈도우 사이즈 내의 값 중에서 하나의 값을 선택하여 압축 


In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instru

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)

앞서 만든 모델에서는 28*28=784 짜리 차원 하나로 구성했지만, CNN 모델에서는 2차원 평면으로 구성하므로 위 처럼 좀 더 직관적인 형태로 가능하다. 

In [7]:
# 1st layer
# Convolution Layer
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
C1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
C1 = tf.nn.relu(C1)

# Pooling Layer
L1 = tf.nn.max_pool(C1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

print('X : ', X)
print('C1 : ', C1)
print('L1 : ', L1)


X :  Tensor("Placeholder_3:0", shape=(?, 28, 28, 1), dtype=float32)
C1 :  Tensor("Relu_3:0", shape=(?, 28, 28, 32), dtype=float32)
L1 :  Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)


### Convolution Layer

- 3 x 3 크기의 커널을 32개 만듬
- 오른쪽과 아랫쪽으로 한칸씩 움직이는(stride) 32개의 커널을 가짐
- `padding='SAME'` : 커널 슬라이딩 시 이미지의 가장 외곽에서 한칸 밖으로 움직이는 옵션, 테두리도 좀더 명확하게 평가 가능

### Pooling Layer
- `ksize=[1,2,2,1]` : 커널 크기가 2 x 2인 풀링 계층을 만든다.
- `strides=[1,2,2,1]` : 커널 슬라이딩시 두칸씩 움직이겠다는 옵션



In [8]:
# 2nd Layer
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
C2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
C2 = tf.nn.relu(C2)
L2 = tf.nn.max_pool(C2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

print('C2 : ', C2)
print('L2 : ', L2)


C2 :  Tensor("Relu_4:0", shape=(?, 14, 14, 64), dtype=float32)
L2 :  Tensor("MaxPool_3:0", shape=(?, 7, 7, 64), dtype=float32)


- `[3, 3, 32, 64]` : 32 - 앞서 구성한 컨볼루션 커널 개수, 64 - 현재 레이어 커널개수

In [9]:
# 3rd Layer
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

1. 10개 분류는 1차원 배열(0~9)이므로 차원을 줄이는 단계를 거쳐야합니다.
2. 직전 풀링 계층의 크기가 7*7*64 이므로, 먼저 tf.reshape 함수를 이용해 1차원 계층으로 만들고 256개의 뉴런으로 연결되는 신경망을 만들어준다.

참고로 이처럼 인접한 계층의 모든 뉴런과 상호 연결된 계층을 완전 연결 계층 `fully connected layer`라고 한다.

In [10]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
opt = tf.train.AdamOptimizer(0.001).minimize(cost)
# opt = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

print('batch size : %s' % batch_size)
print('total batch size : %s' % total_batch)

for epoch in range(10):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([opt, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1), 'Avg. cost = {:.3f}'.format(total_cost/total_batch))

print('Complete!')

batch size : 100
total batch size : 550
Epoch: 0001 Avg. cost = 0.330
Epoch: 0002 Avg. cost = 0.102
Epoch: 0003 Avg. cost = 0.071
Epoch: 0004 Avg. cost = 0.053
Epoch: 0005 Avg. cost = 0.044
Epoch: 0006 Avg. cost = 0.037
Epoch: 0007 Avg. cost = 0.031
Epoch: 0008 Avg. cost = 0.027
Epoch: 0009 Avg. cost = 0.024
Epoch: 0010 Avg. cost = 0.021
Complete!


- `batch_xs = batch_xs.reshape(-1, 28, 28, 1)` : 입력데이터를 재구성

In [16]:
is_correct = tf.equal(tf.argmax(model, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy : %s' % sess.run(accuracy, 
                                 feed_dict={X: mnist.test.images.reshape(-1,28,28,1), Y: mnist.test.labels, keep_prob: 1}))

Accuracy : 0.9882


- `mnist.test.images.reshape(-1,28,28,1)` : 입력데이터 재구성